# Lab01: Frequent itemset mining

- Student ID: 18127070 
- Student name: Trần Đại Chí

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    return data, s

### FP - Growth

**I gave you pseudo code of Apriori algorithm above but we implement FP-Growth. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a,b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space) 

    -------------------
    return 
        ret: itemset generated by joining a and b
    '''
    #TODO (hint: this function will be called in generateSearchSpace method.):
    ret = list(set(a) | set(b))
    return ret
    
class FP_Growth:
    def __init__ (self, data=None, s=None, minSup=None):
        self.data=data
        self.s={}
        
        
        for key, support in sorted(s.items(),key= lambda item: item[1]):
            self.s[key]=support
        #TODO: why should we do this, answer it at the markdown below?
           
        self.minSup=minSup
        self.L={}  #Store frequent itemsets mined from database
        self.runAlgorithm()
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree={}

        search_space={}  
        for item, support in self.s.items():
            search_space[item]={}
            
            search_space[item]['itemset']=[item] 
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            
            search_space[item]['pruned']=False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this 
            # instead of delete item directly from search_space and tree. 
            
            search_space[item]['support']=support
            
            tree[item]={}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            ''' 
        
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        #TODO (hint: this is why i use python set in data)
        support = self.s[itemset] / len(self.data)
        return support
    
    
        
    def prune(self,k, tree, search_space):
        '''
        In this method we will find out which itemset in current search space is frequent 
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets. 
        '''
        #TODO
        items = list(tree.keys())
        for item in items:
            support = self.computeItemsetSupport(item)
            compute_sup = self.minSup / len(self.data)
            if support >= compute_sup:
                self.L[k].append(item)
                search_space[item]['pruned'] = False
            else:
                self.L[k] = []
                search_space[item]['pruned'] = True  
        
        
                
    def generateSearchSpace(self,k, tree, search_space):
        if k <= 0: return
        '''
        Generate search space for exploring k+1 itemset. (Recursive function) 
        '''
        
        items=list(tree.keys())  
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
         
    
        l=len(items)
        self.prune(k, tree, search_space)
        if l==0: return   #Stop condition
        
        
        
        for i in range(l-1):
            a=items[i]
            if search_space[a]['pruned']: continue 
                
            for j in range(i+1,l):
                b=items[j]
                
                search_space[a][b]={}
                tree[a][b]={}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft). 
                
                #TODO:
                #First create newset using join set
                new_l1, new_l2 = [], []
                new_l1.append(a)
                new_l2.append(b)
                new_set = joinset(new_l1, new_l2)
                new_set = set(new_set)
                
                #Second add newset to search_space 
                search_space[a][b] = new_set
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k - 1, tree, search_space)
    
    def runAlgorithm(self):
        tree,search_space=self.initialize() #generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
    def miningResults(self):
        print("Frequent itemset L1\n")
        return self.L
        

Ok, let's test FP-growth on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')

In [5]:
a=FP_Growth(data=data,s=s, minSup=3000)
print(a.miningResults())

Frequent itemset L1

{1: [48, 56, 66, 34, 62, 7, 36, 60, 40, 29, 52, 58]}


### Answer questions here:
1) 
- self.s[key] = support, we do this to get support corresponds to each itemset in our data

2) 
- we use set instead of list because in step joinset if we want to join a and b without duplicate we need to use set,
- example a = ABC, b = CEF, a | b = ABCEF and the major advantage of using a set, as opposed to a list is that it has a highly
- optimized method for checking whether a specific element is contained in the set and if our data is big enough, a set
- is faster for containing checks than list

3) 
- As we mentioned above, a set is faster than a list when checking if an item is contained within it, so we will check 
- rapidly itemsets that wasn't pruned 

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). You can download dataset from here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt. Write your report and implementation below.

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  
More information about `churn` here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir4/Churn.pdf)*

**TODO:**

In [6]:
def readFile():
    f = open("churn.txt", 'r')
    for line in f:
        data = line.split(',')
        data[0] = "State: " + data[0]
        data[1] = "Account Length: " + data[1]
        data[2] = "Area Code: " + data[2]
        data[3] = "Phone: " + data[3]
        data[4] = "Int'l Plan: " + data[4]
        data[5] = "VMail Plan: " + data[5]
        data[6] = "VMail Message: " + data[6]
        data[7] = "Day Mins: " + data[7]
        data[8] = "Day Calls: " + data[8]
        data[9] = "Day Charge: " + data[9]
        data[10] = "Eve Mins: " + data[10]
        data[11] = "Eve Calls: " + data[11]
        data[12] = "Eve Charge: " + data[12]
        data[13] = "Night Mins: " + data[13]
        data[14] = "Night Calls: " + data[14]
        data[15] = "Night Charge: " + data[15]
        data[16] = "Intl Mins: " + data[16]
        data[17] = "Intl Calls: " + data[17]
        data[18] = "Intl Charge: " + data[18]
        data[19] = "CustServ Calls: " + data[19]
        data[20] = "Churn?: " + data[20]
        if not data: break
        yield data

data = readFile()

class ChurnAnalysis:
    def __init__(self):
        self.minSup = 2000 #0.6
        self.rowTransaction = set() #all data of each row in dataset
        self.L = defaultdict(int) #store number of a itemset for computing its support
    def candidateC1(self):
        itemSet = set()
        for i in data:
            tid = frozenset(i)
            self.rowTransaction.add(tid)
            for item in tid: itemSet.add(frozenset([item])) # use frozenset as a key in a dict
        sorted(itemSet, reverse=False)
        return itemSet
    def itemWithMinsup(self, Ck):
        prune = set() #store itemset satisfy minSup
        cnt = defaultdict(int)
        for tid in self.rowTransaction:
            for item in Ck:
                if item.issubset(tid):
                    self.L[item] += 1
                    cnt[item] += 1
        for key, support in cnt.items():
            sup = support / len(self.rowTransaction)
            convertMinsup = round(self.minSup / len(self.rowTransaction), 2)
            if sup >= convertMinsup: prune.add(key)
            else: continue 
        return prune
    def computeSupport(self, item):
        return self.L.get(item) / len(self.rowTransaction)
    def selfjoin(self, itemset):
        Ck = set()
        for a in itemset:
            for b in itemset:
                sorted(a, reverse=False)
                sorted(b, reverse=False)
                c = a & b 
                if(len(c) == len(a) - 1): Ck.add(a | b)
        return Ck
    def apriori(self):
        aprioriGen = dict()
        C1 = self.candidateC1()
        Lk = self.itemWithMinsup(C1)
        k = 2
        if(len(Lk) == 0): return
        while(len(Lk) != 0):
            aprioriGen[k - 1] = Lk
            new_join = self.selfjoin(Lk)
            scanDB = self.itemWithMinsup(new_join)
            Lk = scanDB
            k += 1
        result = []
        for key, value in aprioriGen.items():
            result.extend([(set(item), self.computeSupport(item)) for item in value])
        for key, support in sorted(result, key=lambda item: item[1]):
            print(str(key) + ' with support: ' + str(round(support, 2)))

def main():
    a = ChurnAnalysis()
    a.apriori()

if __name__ == '__main__':
    main()

{'VMail Message: 0', 'Churn?: False.\n'} with support: 0.6
{'VMail Plan: no', 'Churn?: False.\n'} with support: 0.6
{'VMail Plan: no', 'VMail Message: 0', 'Churn?: False.\n'} with support: 0.6
{"Int'l Plan: no", 'VMail Message: 0'} with support: 0.65
{"Int'l Plan: no", 'VMail Plan: no'} with support: 0.65
{"Int'l Plan: no", 'VMail Plan: no', 'VMail Message: 0'} with support: 0.65
{'VMail Message: 0'} with support: 0.72
{'VMail Plan: no'} with support: 0.72
{'VMail Plan: no', 'VMail Message: 0'} with support: 0.72
{"Int'l Plan: no", 'Churn?: False.\n'} with support: 0.8
{'Churn?: False.\n'} with support: 0.85
{"Int'l Plan: no"} with support: 0.9


- First, i read and handle the file, then i use apriori algorithm for analysing churn purpose 
- candidateC1(self): in this function, we will generate all candidate C1 from out dataset. We use frozenset instead of set because we will later use these sets as the key in the dictionary, so we need to make it immutable
- itemWithMinsup(self, Ck): in this function, we will have list frequent L1 from list of candidate C1 we created from function candidateC1(self), then we take itemsets satisfy our minSup
- computeSupport(self, item): calculate support of a item in dataset
- selfjoin(self, itemset): this function will take list of frequent itemset Lk to produce Ck. Example our L1 is  {a}, {b}, {c} -> {a, b}, {a, c}, {b, c} and two set are combined using union which is the "|" symbol
- apriori(self): main function for running apriori algorithm and print all frequent itemsets Lk with their support that satisfy our minSup
- As we can see from result above, we have total 12 frequent itemsets with minSup >= 2000

# 4 References
http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/ProjectDefinition1.pdf

https://cs.wmich.edu/~alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part1.pdf

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
